In [ ]:
import pandas as pd
from gensim import corpora, models
from gensim.parsing.preprocessing import preprocess_documents
from bs4 import BeautifulSoup
from glob import glob
import os
import numpy as np
from preprocessing import get_key_words
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gm
import time

## Pandas stuff

In [ ]:
df = pd.read_csv('financial_services_rules.csv', usecols=
                 ['document_number', 'publication_date'])
df['publication_date']= pd.to_datetime(df['publication_date'])
df = df.sort_values(by='publication_date')
print(df.shape)
df.reset_index(inplace=True, drop=True)
df.head(5)

(3630, 2)


,document_number,publication_date
0,01-43,2001-01-03
1,01-34,2001-01-03
2,01-42,2001-01-03
3,01-175,2001-01-05
4,01-595,2001-01-09


In [ ]:
def myfun(s):
    return os.path.exists(os.path.join(r"./docs", s['document_number'] + ".xml"))

df = df[df.apply(myfun, axis=1)]
# date_df = df[df['publication_date'] < "2007-01-01"].copy()
df.head()

,document_number,publication_date
0,01-43,2001-01-03
1,01-34,2001-01-03
2,01-42,2001-01-03
3,01-175,2001-01-05
4,01-595,2001-01-09


In [ ]:
# Save
id2date = dict(zip(df.document_number, df.publication_date))
id2date
# np.save('id2date.npy', id2date) 

In [ ]:
# def preprocess_file(filename : str, MIN_LENGTH : int):
#     infile = open(filename, encoding="utf8")
#     contents = infile.read()
#     soup = BeautifulSoup(contents, 'xml')
#     p_tags = soup.find_all("P")
    
#     # Filter out shortened phrases and then preprocess
#     filter_short_phrases = [phrase.get_text() for phrase in p_tags if len(phrase.get_text()) > MIN_LENGTH]
#     kw_mat = preprocess_documents(filter_short_phrases)
    
#     # Flatters 2D List into 1D
#     return [word for kws in kw_mat for word in kws]

In [ ]:
# files = glob("./docs/*")
# print(len(files))

# MIN_LENGTH = 300 # chosen parameter

# for idx, file in enumerate(files):
#     if idx % 500 == 0:
#         print(idx)
#     # Extract documentID from filepath
#     basename = os.path.basename(file)
#     documentId = os.path.splitext(basename)[0]
#     if documentId not in id2date:
#         print(documentId)
#         continue
        
#     # String of preprocessed words in given file
#     words = preprocess_file(file, MIN_LENGTH)
    
#     np.save('{}_words.npy'.format(documentId), words)
    

# Visualizations

In [ ]:
path = r"./docs" # Change this to the path of your /docs folder on your system

processed_docs = {}

for filename in glob(os.path.join(path, "*.xml")):
    processed_docs[filename] = get_key_words(filename)
np.save("processed_docs", processed_docs)

# for i,row in df_exists[df_exists['publication_date'] >= "2007-01-01"].iterrows():
#     filename = os.path.join(path, row['document_number'] + ".xml")
#     processed_docs.append(get_key_words(filename))

# dictionary = gensim.corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)

In [ ]:
type(processed_docs)

dict

# Pre 2007 Topics

In [ ]:
filtered_docs = []

for doc in df[df['publication_date'] < "2007-01-01"]['document_number'].tolist():
    filename = os.path.join(path, doc + ".xml")
    filtered_docs.append(processed_docs[filename])

In [ ]:
dictionary = corpora.Dictionary(filtered_docs)
dictionary.filter_extremes(no_below=5, no_above=0.7, keep_n= 100000)

num_topics = 5
corpus = [dictionary.doc2bow(text) for text in filtered_docs]
lda_model = LdaModel(corpus, num_topics=num_topics)

vis = gm.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\leero\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


# Post 2007 Topics



In [ ]:
filtered_docs = []

for doc in df[df['publication_date'] >= "2007-01-01"]['document_number'].tolist():
    filename = os.path.join(path, doc + ".xml")
    filtered_docs.append(processed_docs[filename])

In [ ]:
dictionary = corpora.Dictionary(filtered_docs)
dictionary.filter_extremes(no_below=5, no_above=0.70, keep_n= 100000)

num_topics = 5
corpus = [dictionary.doc2bow(text) for text in filtered_docs]
lda_model = LdaModel(corpus, num_topics=num_topics)

vis = gm.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\leero\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


# Obama Admin

In [ ]:
filtered_docs = []

for doc in df[(df['publication_date'] >= "2008-01-01") & (df['publication_date'] < "2009-12-31")]['document_number'].tolist():
    filename = os.path.join(path, doc + ".xml")
    filtered_docs.append(processed_docs[filename])
    
print(df.shape)
print(len(filtered_docs))

(3623, 2)
325


In [ ]:
dictionary = corpora.Dictionary(filtered_docs)
dictionary.filter_extremes(no_below=5, no_above=0.70, keep_n= 100000)

num_topics = 5
corpus = [dictionary.doc2bow(text) for text in filtered_docs]
lda_model = LdaModel(corpus, num_topics=num_topics)

vis = gm.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\leero\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


# Trump Admin

In [ ]:
filtered_docs = []

for doc in df[(df['publication_date'] >= "2016-01-01") & (df['publication_date'] < "2017-12-31")]['document_number'].tolist():
    filename = os.path.join(path, doc + ".xml")
    filtered_docs.append(processed_docs[filename])
    
print(df.shape)
print(len(filtered_docs))

(3623, 2)
318


In [ ]:
dictionary = corpora.Dictionary(filtered_docs)
dictionary.filter_extremes(no_below=5, no_above=0.70, keep_n= 100000)

num_topics = 5
corpus = [dictionary.doc2bow(text) for text in filtered_docs]
lda_model = LdaModel(corpus, num_topics=num_topics)

vis = gm.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\leero\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
